In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error


In [ ]:
def tensorizing(window_size,no_target_data):
    '''
    input:
    window_size = number of records in each segment
    no_target_data = data withot target
    output:
    tensor_data= stacked segments with step 1 frontly(mode3)
    '''
    #this func transform the 2D data to a 3D tensor
    # first dimension is timeseries samples , second is features and third dimension is time
    tensor_data=np.zeros((no_target_data.shape[0]-window_size+1,window_size,no_target_data.shape[1]))
    for i in range(no_target_data.shape[0]-window_size+1):
        tensor_data[i,:,:]=no_target_data.iloc[i:i+window_size].to_numpy()
    print ("data without target size is",no_target_data.shape)
    print ("tensorized data size is",tensor_data.shape)
    return tensor_data

In [ ]:
windowsize=30
x_train, x_test, y_train, y_test = train_test_split(pd.DataFrame(X_pca), Y_datacut, test_size = 500,shuffle=False)
scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(x_train)
x_train=scaler.transform(x_train)
x_train=tensorizing(windowsize,pd.DataFrame(x_train))
# x_train=tensorizing(windowsize,x_train)

y_train=y_train[windowsize-1:]

In [ ]:
model = Sequential()
model.add(LSTM(50,return_sequences = True, input_shape=( windowsize, x_train.shape[2])))

model.add(LSTM(50, activation='linear'))
# model.add(LSTM(50, activation='sigmoid'))

my_adam_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.add(Dense(1,activation='linear'))
model.compile(loss='mean_squared_error', optimizer=my_adam_optimizer)
model.fit(x_train, y_train, epochs=70,batch_size=32)

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(x_test)
x_test=scaler.transform(x_test)
x_test=tensorizing(windowsize,pd.DataFrame(x_test))
# x_test=tensorizing(windowsize,x_test)

y_test=y_test[windowsize-1:]

In [ ]:
mse = model.evaluate(x_test, y_test)


In [ ]:
yhat = model.predict(x_test)
rmse=np.sqrt(metrics.mean_squared_error(y_test, yhat))
rmse